In [64]:
# Importing Data
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE

In [65]:
# Load dataset
data = pd.read_csv('creditcard.csv')

In [66]:
# Feature Scaling
scaler = MinMaxScaler()
data.iloc[:, :-1] = scaler.fit_transform(data.iloc[:, :-1])

In [67]:
# Splitting data
X = data.iloc[:, :-1]
y = data['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [68]:
# Handle imbalance using SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
X_train_sampled = X_train_resampled.sample(n=5000, random_state=42)  # Reduce to 5000 samples
y_train_sampled = y_train_resampled[X_train_sampled.index]

In [69]:
# Train SVM
svm = SVC(kernel='rbf', random_state=42)
svm.fit(X_train_sampled, y_train_sampled)
y_pred_svm = svm.predict(X_test)

In [70]:
# Train Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_sampled, y_train_sampled)
y_pred_rf = rf.predict(X_test)

In [71]:
# Evaluate models
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
svm_scores = [
    accuracy_score(y_test, y_pred_svm) * 100,
    precision_score(y_test, y_pred_svm) * 100,
    recall_score(y_test, y_pred_svm) * 100,
    f1_score(y_test, y_pred_svm) * 100
]

rf_scores = [
    accuracy_score(y_test, y_pred_rf) * 100,
    precision_score(y_test, y_pred_rf) * 100,
    recall_score(y_test, y_pred_rf) * 100,
    f1_score(y_test, y_pred_rf) * 100
]

print("SVM:")
for metric, svm_score in zip(metrics, svm_scores):
    print(f"{metric}: {svm_score:.2f}%")

print("\n")
print("Random Forest:")
for metric, rf_score in zip(metrics, rf_scores):
    print(f"{metric}: {rf_score:.2f}%")

SVM:
Accuracy: 99.03%
Precision: 13.90%
Recall: 88.78%
F1-Score: 24.03%


Random Forest:
Accuracy: 99.46%
Precision: 22.72%
Recall: 88.78%
F1-Score: 36.17%


In [72]:
# Create a DataFrame with metrics as headers
scores_table = pd.DataFrame(
    [svm_scores, rf_scores],
    columns=metrics,
    index=['SVM', 'Random Forest']
)
scores_table = scores_table.round(2)
scores_table = scores_table.map(lambda x: f"{x:.2f}%")

# Display the table
display(scores_table)

,Accuracy,Precision,Recall,F1-Score
SVM,99.03%,13.90%,88.78%,24.03%
Random Forest,99.46%,22.72%,88.78%,36.17%
